In [1]:
import tweepy as tw
import pandas as pd
import numpy as np

### Authentication

In [ ]:
import twtr_credentials  #keys and tokens file

In [ ]:
auth = tw.OAuthHandler(twtr_credentials.API_key, twtr_credentials.API_key_secret)
auth.set_access_token(twtr_credentials.Access_token, twtr_credentials.Access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

### Get Tweets

In [ ]:
hashtag = '#bitcoin'

In [ ]:
query = tw.Cursor(api.search_tweets, q=hashtag).items(1000)
tweets = [{'Tweets':tweet.text, 'Timestamp':tweet.created_at} for tweet in query]

In [ ]:
df = pd.DataFrame(tweets)

### Preprocessing

In [ ]:
import nltk
from nltk.corpus import stopwords
from textblob import Word, TextBlob

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
cust_words = ['RT','#bitcoin']

In [ ]:
def preporcess_tweets(tweet, cust_words):
    preprocessed_tweet = tweet
    preprocessed_tweet.replace('[^\w\s]','')
    preprocessed_tweet = ' '.join(word for word in preprocessed_tweet.split() if word not in stop_words)
    preprocessed_tweet = ' '.join(word for word in preprocessed_tweet.split() if word not in cust_words)
    preprocessed_tweet = ' '.join(Word(word).lemmatize() for word in preprocessed_tweet.split())
    return preprocessed_tweet

In [ ]:
df['Processed Tweet'] = df['Tweets'].apply(lambda x: preporcess_tweets(x, cust_words))

### Sentiment Analysis

In [ ]:
df['Polarity'] = df['Processed Tweet'].apply(lambda x: TextBlob(x).sentiment[0])
df['Subjectivity'] = df['Processed Tweet'].apply(lambda x: TextBlob(x).sentiment[1])

In [ ]:
df.describe()

### Visualize

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('fivethirtyeight')

In [ ]:
df['MA Polarity'] = df.Polarity.rolling(10, min_periods = 3).mean()

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(df['Timestamp'],df['MA Polarity'])